In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, date
import pickle
from time import time
from src.utils import libSort
import calendar
from datetime import datetime as dpa
from calendar import monthrange
from src.utils.convert_data_type import convertList, convertToHour
from src.utils.QueryData import GetPhone, checkNumberDayRemaining, SearchInfor, GetMinute, GetIndex
from src.utils.mergeToDataFrame import format_time

database = pd.read_csv('/home/ai_car/AthenaBow/data/TotalData.csv')
xgbModel = pickle.load(open('/home/ai_car/AthenaBow/model/24-12/xgb.sav', 'rb'))
# database = database.drop(columns=['STT', 'CallID', 'Hotline', 'QC User Say', 'QC Intent', 'Đánh giá Intent', 'Tỷ lệ nhận diện STT', 'Đánh giá STT', 'Audio', 'Đơn giá', 'CUSTOMER'])
# database = database.drop(columns=['Intent', 'Độ tự tin (%)', 'Entities', 'Entity Value', 'QC Độ trễ', 'Số khách hàng', 'Mã kết thúc bot'])
# database = database.drop(columns=['Action', 'Bot Process (ms)', 'TTS Process (ms)', 'User Say', 'Time in call', 'ASR Process (ms)'])
# database = database.drop(columns=['TENURE', 'RELATIONSHIP_2', 'PHONE_REF3', 'NAME_3', 'RELATIONSHIP_3', 'PERMNENT_PROVINCE', 'RELATIONSHIP_1', 'PHONE_REF2', 'NAME_2'])
# database = database.drop(columns=['MOB', 'DPD', 'REMAINING_PEROID'])
# database = database.drop(columns=['NAME_1', 'DISBURSALDATE', 'PHONE_REF1', 'MAX_DPD', 'DUEDAY', 'FIRST_PAID_DATE'])
# database = database.drop(columns=['Mã tổng đài', 'AMOUNT_FINANCE', 'GENDER', 'Thời lượng gọi'])
# database = database.drop_duplicates(ignore_index= True)
# database = database.dropna(axis=1)
# database = database.reset_index(drop=True)
def convertPhone(database):
    listPhone = []
    for i in tqdm(range(len(database['CURRES_PHONE']))):
        if str(database['CURRES_PHONE'][i]) == "nan":
            listPhone.append(0)
        else:
            listPhone.append(int(database['CURRES_PHONE'][i]))
    return listPhone
database['CURRES_PHONE'] = convertPhone(database)
def encodeTimeCall(data):
    listEncode = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        a = dpa.strptime(data['Thời điểm gọi'][i], '%Y-%m-%d %H:%M:%S')
        listEncode.append(a)
    return listEncode
database['Thời điểm gọi'] = encodeTimeCall(database)
def convertToTime(data):
    hourToCall = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        hourToCall.append(convertToHour(data, i))
    return hourToCall
database['TimeCall'] = convertToTime(database)
listTimeEncode = []
for i in range(len(database['TimeCall'])):
    listTimeEncode.append(int(int(database['TimeCall'][i]) * 60) + GetMinute(database, i))
database['TimeCall'] = listTimeEncode
database.rename(columns = {'Trạng thái':'Result'}, inplace = True)
def labelEncoder(data):
    encoderLabel = []
    for i in tqdm(range(len(data['Result']))):
        if data['Result'][i] == 'Không thành công' or data['Result'][i] == 'Lỗi':
            encoderLabel.append(0)
        elif data['Result'][i] == 'Nghe máy':
            encoderLabel.append(1)
    return encoderLabel

database['Result'] = labelEncoder(database)

/home/ai_car/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (8,9,13,14,15,17,18,25,36,37,40,43,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 1699122/1699122 [00:18<00:00, 94317.03it/s] 


In [14]:
listAgreedmentID = 2523722
                    # 2662516,
                    # 1661663,
                    # 2599746,
                    # 2421024]

listAgreedmentID = 2014401
# for i in range(len(listAgreedmentID)):
listID = GetIndex(data= database, id= listAgreedmentID)
Phone, Product, Nc = SearchInfor(data= database, index= listID[0], NCIndex= listID[-1])
historyCall = GetPhone(data=database, Phone= Phone)
NowDay, LimitInMonth, NowMonth = checkNumberDayRemaining()
whatInsideToModel = [[Phone, Product, Nc]]
listResult = pd.DataFrame()
for i in tqdm(range(len(whatInsideToModel))):
    for j in range(NowDay, LimitInMonth + 1):
        perCent = []
        listTimeAvailabel = []
        for k in tqdm(range(480, 12001, 15)):
            preds = xgbModel.predict_proba([[whatInsideToModel[i][0],
                                            whatInsideToModel[i][1],
                                            whatInsideToModel[i][2],
                                            k,
                                            j,
                                            NowMonth]])
            preds = preds.tolist()
            perCent.append(preds[0][1])
            listTimeAvailabel.append(k)
        whenIsGoodTime = libSort.nlargest(10,zip(perCent, listTimeAvailabel))
        whenIsGoodTime = convertList(whenIsGoodTime)
        StandardScaleTime = []
        for ff in range(len(whenIsGoodTime)):
            HereWeLive = (f"{round(float(whenIsGoodTime[ff][0] * 100),3)}", whenIsGoodTime[ff][1])
            StandardScaleTime.append(HereWeLive)
        DayinWeeks = date(year=datetime.date(datetime.now()).year,
                            month=NowMonth,
                            day= int(j))
        listResult[f'{j}/{NowMonth}/{datetime.date(datetime.now()).year} - {calendar.day_name[DayinWeeks.weekday()]}'] = StandardScaleTime


100%|██████████| 1/1 [00:09<00:00,  9.97s/it]


In [15]:
maxPercent = float(90)
Statictis = []
for m in listResult.columns:
    countTotals = 0
    for n in range(len(listResult[m])):
        countReach = 0
        countReachInTime = 0
        if float(listResult[m][n][0]) < maxPercent:
            pass
        else:
            extractDay = int(m.split('/')[0])
            for p in range(len(historyCall)):
                if int(database['Thời điểm gọi'][historyCall[p]].day) == extractDay:
                    countTotals = countTotals + 1
                    if int(listResult[m][n][1]) in range(int(database['TimeCall'][historyCall[p]]) + 15,int(database['TimeCall'][historyCall[p]]) + 15):
                        if int(database['Result'][historyCall[p]]) == 1:
                            countReachInTime = countReachInTime + 1
                            countReach = countReach + 1
                        else:
                            pass
                    else:
                        if int(database['Result'][historyCall[p]]) == 1:
                            countReach = countReach + 1
        try:
            BestofDay = {'Time to Call' : format_time(listResult[m][n][1]),
                        'Total Call' : countTotals,
                        'Total Reach In Day' : countReach,
                        'Total Reach In Time' : countReachInTime,
                        'Confident' : countReachInTime/countTotals}
        except:
            raise Exception("Not any data of ID")
    ResultIHope = {str(m) : BestofDay}
    Statictis.append(ResultIHope)

Exception: Not any data of ID

In [2]:
import pandas as pd

dt = pd.read_csv('/home/ai_car/AthenaBow/nhancuachungta/ahihi.csv')
dt.rename(columns = {'Trạng thái':'Result'}, inplace = True)

,Unnamed: 0,0,0.1,1,2,3,4,5
0,0,27/12/2022 - Tuesday,11h30,7,5,71.428571,5,88.622
1,1,27/12/2022 - Tuesday,11h15,14,10,71.428571,10,88.183
2,2,27/12/2022 - Tuesday,10h30,21,15,71.428571,15,88.124
3,3,27/12/2022 - Tuesday,11h00,28,20,71.428571,20,87.893
4,4,27/12/2022 - Tuesday,10h45,35,25,71.428571,25,87.775
5,5,27/12/2022 - Tuesday,10h00,42,30,71.428571,30,87.594
6,6,27/12/2022 - Tuesday,9h00,49,35,71.428571,35,87.467
7,7,27/12/2022 - Tuesday,8h45,56,40,71.428571,40,87.467
8,8,27/12/2022 - Tuesday,9h15,63,45,71.428571,45,87.354
9,9,27/12/2022 - Tuesday,9h45,70,50,71.428571,50,87.261


In [5]:
dt['Unnamed: 0']

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
24    24
25    25
26    26
27    27
28    28
29    29
30    30
31    31
32    32
33    33
34    34
35    35
36    36
37    37
38    38
39    39
40    40
41    41
42    42
43    43
44    44
45    45
46    46
47    47
48    48
49    49
Name: Unnamed: 0, dtype: int64

In [4]:
dt.value_counts()

Unnamed: 0  0                       0.1    1    2   3          4   5     
0           27/12/2022 - Tuesday    11h30  7    5   71.428571  5   88.622    1
37          30/12/2022 - Friday     10h00  32   8   25.000000  8   88.349    1
27          29/12/2022 - Thursday   8h45   56   40  71.428571  40  87.846    1
28          29/12/2022 - Thursday   9h15   63   45  71.428571  45  87.540    1
29          29/12/2022 - Thursday   9h45   70   50  71.428571  50  87.441    1
30          30/12/2022 - Friday     11h15  4    1   25.000000  1   89.601    1
31          30/12/2022 - Friday     11h30  8    2   25.000000  2   89.502    1
32          30/12/2022 - Friday     11h00  12   3   25.000000  3   89.315    1
33          30/12/2022 - Friday     10h30  16   4   25.000000  4   89.063    1
34          30/12/2022 - Friday     10h45  20   5   25.000000  5   88.889    1
35          30/12/2022 - Friday     10h15  24   6   25.000000  6   88.559    1
36          30/12/2022 - Friday     9h45   28   7   25.00